In [ ]:
"""
Working with checklists.

This notebook show you how to fetch checklists from the database.

The records loaded from spreadsheet rows or returned from the eBird API
are saved across five different database tables: Checklist, Location, 
Observer, Observation and Species, which saves a lot of space, but to 
load all the fields for a checklist (not the observations) you need 
to join the tables back together. That's easy to do with SQLAlchemy, 
however eBird Notebooks also has a helper class, Checklists, which 
implements the most commonly used database queries. 

Each row fetched contains three objects: Checklist, Location, and 
Observation. You access the fields 'dot operator' used to access an 
object's methods or attributes. For example:

    row.Checklist.date
    row.Location.state
    row.Observer.name

This notebook loads the checklists from a random country into a database
stored in memory - so there's no special setup. The each cell shows you 
how to use the Checklists class to fetch the records.

For a full list of all the fields available, see the ebird.notebooks.models
file in the 'src' directory of this project.
"""

import datetime as dt

from faker import Faker
from IPython.display import display, HTML
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.pool import StaticPool

from ebird.notebooks import models, settings, loaders
from ebird.notebooks.readers import Checklists

# Get the key for accessing the eBird API.
api_key = settings.API_KEY

# Create the database for the notebook in memory.
# Normally you just pass the connection string (URL). The other arguments
# are only needed because the database is stored in memory, They are not
# needed when accessing a file or database server.
engine = create_engine("sqlite:///", connect_args={'check_same_thread':False}, poolclass=StaticPool)

In [ ]:
# Create the database tables.
models.Base.metadata.create_all(engine)

In [ ]:
# The eBird API returns only the species code, e.g. "horlar1", so
# we need to populate the species table with the complete taxonomy.
with Session(engine) as session:
    loader = loaders.SpeciesLoader(api_key, session)
    loader.load()

In [ ]:
# Select a country at random.
country_code = Faker().country_code()

# Load the 10 most recent checklists from the eBird API.
with Session(engine) as session:
    loader = loaders.APILoader(api_key, session)
    loader.load(country_code, max_results=10)

In [ ]:
# Get the Location from the most recent Checklist to get some values
# for the example code in the following cells.

with Session(engine) as session:
    latest = Checklists(session).latest()

# (Ab)use the walrus operator, so the code is compact.
print("Country: ", country := latest.Location.country)
print("Country code: ", country_code := latest.Location.country_code)
print("State: ", state := latest.Location.state)
print("State code: ", state_code := latest.Location.state_code)
print("County: ", county := latest.Location.county)
print("County code: ", county_code := latest.Location.county_code)

# More checklists should have been submitted yesterday
date = latest.Checklist.date - dt.timedelta(days=1)

In [ ]:
# Display the observations in a table

header_row = '''
  <tr>
    <th style="text-align: left">Identifier</th>
    <th style="text-align: left">Date</th>
    <th style="text-align: left">Time</th>
    <th style="text-align: left">Location</th>
  </tr>
'''

body_row = '''
  <tr>
    <td style="text-align: left">{}</td>
    <td style="text-align: left">{}</td>
    <td style="text-align: left">{}</td>
    <td style="text-align: left">{}</td>
  </tr>
'''    

def show_table(rows, limit=None):
    if limit:
        rows = list(rows)[:limit]

    data = [
        [
            row.Checklist.identifier,
            row.Checklist.date,
            row.Checklist.time,
            "{}, {}, {}".format(row.Location.name, row.Location.state, row.Location.country)
        ] for row in rows
    ]

    body = [body_row.format(*row) for row in data]
    table = "<table><thead>{}</thead><tbody>{}</tbody></table>".format(header_row, "".join(body))
    
    return display(HTML(table))

In [ ]:
# Fetch every checklist.
with Session(engine) as session:
    rows = Checklists(session).fetch()
    table = show_table(rows)

table

In [ ]:
# Count the number of checklists.
with Session(engine) as session:
    count = Checklists(session).for_year(date.year).count()

    print("Checklists submitted in %d: " % date.year, count)

In [ ]:
# Fetch the checklists for a given country.

In [ ]:
# Using the name of the country.
with Session(engine) as session:
    rows = Checklists(session).for_country(country).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Or using the country code.
with Session(engine) as session:
    rows = Checklists(session).for_country(country_code).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Fetch the checklists for a given state (subnational1 area).

In [ ]:
# Using the name of the state.
with Session(engine) as session:
    rows = Checklists(session).for_state(state).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Or using the code for the state.
with Session(engine) as session:
    rows = Checklists(session).for_state(state_code).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Fetch the checklists for a given region.
# This is just an alias for fetching checklists by state.

In [ ]:
# Using the name of the region
with Session(engine) as session:
    rows = Checklists(session).for_region(state).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Or using the code for the region.
with Session(engine) as session:
    rows = Checklists(session).for_region(state_code).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Fetch the checklists for a given county.

In [ ]:
# Using the name of the county
with Session(engine) as session:
    rows = Checklists(session).for_county(county).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Or using the code for the county.
with Session(engine) as session:
    rows = Checklists(session).for_county(county_code).fetch()
    table = show_table(rows)

table

In [ ]:
# Fetch the checklists for a given year.
with Session(engine) as session:
    rows = Checklists(session).for_year(date.year).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Fetch the checklists for a given month.
with Session(engine) as session:
    rows = Checklists(session).for_month(date.year, date.month).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Fetch the checklists for a given day.
with Session(engine) as session:
    rows = Checklists(session).for_day(date.year, date.month, date.day).fetch()
    table = show_table(rows, limit=5)

table

In [ ]:
# Fetch the checklists for a given date.
with Session(engine) as session:
    rows = Checklists(session).for_date(date).fetch()
    table = show_table(rows, limit=5)

table